# CAMSHIFT ALGORITHM

The CAMShift (Continuously Adaptive Mean Shift) algorithm is a colour-based object tracking method introduced by Gary Bradski in 1998 to reduce computational complexity of the methods used during that period and to deal with problems such as image noise, distractors

The CAMShift algorithm derived from the mean shift algorithm, which is responsible for finding the centre of the probability distribution of the object to track. The main difference is that CAMShift adjusts itself to the search window size, for example when object sizes are changing as they move closer to or farther from the camera.

***CODE RUN*** - **After Code Run Select the Area in frame with mouse (hold left and drag) which you wanna track**

***QUIT FRAME*** - **Press Esc key**

***Note*** - **Works Fine in Plain background**

In [4]:
import cv2
import numpy as np


xs,ys,ms,ns = 0,0,0,0  #selection.x selection.y
xo,yo=0,0 #origin.x origin.y
selectObject = False
trackObject = 0
def onMouse(event, x, y, flags, prams): 
    global xs,ys,ms,ns,selectObject,xo,yo,trackObject
    if selectObject == True:
        xs = min(x, xo)
        ys = min(y, yo)
        ms = abs(x-xo)
        ns = abs(y-yo)
    if event == cv2.EVENT_LBUTTONDOWN:
        xo,yo = x, y
        xs,ys,ms,ns= x, y, 0, 0
        selectObject = True
    elif event == cv2.EVENT_LBUTTONUP:
        selectObject = False
        trackObject = -1

capture = cv2.VideoCapture(0)
ret,frame = capture.read()
cv2.namedWindow('frame')
cv2.setMouseCallback('frame',onMouse)
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )
while(True):
    ret,frame = capture.read()
    if trackObject != 0:
        hsv =  cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, np.array((0., 30.,10.)), np.array((180.,256.,255.)))
        if trackObject == -1:
            track_window=(xs,ys,ms,ns)
            maskroi = mask[ys:ys+ns, xs:xs+ms]
            hsv_roi = hsv[ys:ys+ns, xs:xs+ms]
            roi_hist = cv2.calcHist([hsv_roi],[0],maskroi,[180],[0,180])
            cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)
            trackObject = 1
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        dst &= mask
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame,[pts],True, 255,2)
        
    if selectObject == True and ms>0 and ns>0:
        cv2.imshow('frame2',frame[ys:ys+ns,xs:xs+ms])
        cv2.bitwise_not(frame[ys:ys+ns,xs:xs+ms],frame[ys:ys+ns,xs:xs+ms])
    cv2.imshow('frame',frame)
    if  cv2.waitKey(10)==27:
        break
cv2.destroyAllWindows()